In [1]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures
import ray as rayt

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

import pyrr 
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import itertools
%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2021-09-29 17:19:09,189 - utils - NumExpr defaulting to 4 threads.


In [7]:
voxel_size = 0.2
downsample = None
sample = 1
show = True
stop = None
mockname = 'single_57_combined'

%time m3s = rayt.main(mockname, voxel_size, downsample=downsample, sample=sample, stop=stop, show=show)


max --> [47, 48, 30]
min --> [0, 0, 0]
FILE: 	 s1300000.npy
size BPC: 	 30000
size new BPC: 	 1


100%|██████████| 1/1 [00:00<00:00,  8.92it/s]


FILE: 	 s1200000.npy
size BPC: 	 30000
size new BPC: 	 1


100%|██████████| 1/1 [00:00<00:00,  6.04it/s]


FILE: 	 s1100000.npy
size BPC: 	 30000
size new BPC: 	 1


...

FILE: 	 s0900000.npy
size BPC: 	 30000
size new BPC: 	 1


100%|██████████| 1/1 [00:00<00:00, 10.36it/s]


FILE: 	 s0400000.npy
size BPC: 	 30000
size new BPC: 	 1


100%|██████████| 1/1 [00:00<00:00, 13.06it/s]

tot vox: 	 72912
voxels hitted: 	 618
Percentage of voxels hitted by beam: 0.01
CPU times: user 3.95 s, sys: 214 ms, total: 4.16 s
Wall time: 4.45 s


In [14]:
m3s.sum()

3539

In [8]:
m3s2 = np.load('/home/omar/myproj/planttech/data/single_57/toy_trees/m3s_tree_0_0.5.npy')
m3s2.shape

(20, 20, 13)

In [132]:
boxes_reached, boxes_stop = rayt.get_all_AABB(boxPR, voxel_size=0.5, stop=1)
AABBc = [pyrr.aabb.centre_point(box).tolist() for box in boxes_reached]
boxes_stop_idx, boxes_stop_dict = rayt.get_all_voxels_in_stop(AABBc, boxes_stop)
m3AABB, m3stop = rayt.build_matrixes(m3s, boxes_stop_idx, boxes_reached, AABB2vgidx)

print(boxes_reached.shape)
print(boxes_stop.shape)
print(np.array(AABBc).shape)
print(m3AABB.shape)
print(m3stop.shape)

marks:  56
marks:  448
marks:  3584
marks:  12544
marks:  0
(16640, 2, 3)
(8, 2, 3)
(16640, 3)
(20, 20, 13)
(20, 20, 13)


In [156]:
key_list = list(boxes_stop_dict.keys())
val_list = list(boxes_stop_dict.values())

AABBc_ = [pyrr.aabb.centre_point(box).tolist() for box in boxes_stop]
idxs = [key_list[val_list.index(i)] for i in AABBc_]

m3_ = np.zeros_like(m3s, dtype=bool)

for idx in idxs[:4]:
    print(idx)
    m3_ |= (m3stop == idx)



print(set(m3stop[m3_]))
print(m3_.sum())

0
1
2
3
{0, 1, 2, 3}
2600


In [166]:
print(m3_.sum())
print(m3s.sum())
keep = (m3_) & (~m3s)
print(np.sum(keep))

2600
141
2556


In [157]:
# for num, i in enumerate(m3AABB[m3_]):
#     print(num, i)
boxes = [box for box in m3AABB[m3_] if pyrr.geometric_tests.ray_intersect_aabb(ray, box) is not None]

In [158]:
boxes

[array([[-3.13640308,  2.59244299,  8.69805169],
        [-2.63640308,  3.09244299,  9.19805169]]),
 array([[-3.13640308,  3.09244299,  8.69805169],
        [-2.63640308,  3.59244299,  9.19805169]]),
 array([[-3.13640308,  3.59244299,  8.19805169],
        [-2.63640308,  4.09244299,  8.69805169]]),
 array([[-3.13640308,  3.59244299,  8.69805169],
        [-2.63640308,  4.09244299,  9.19805169]]),
 array([[-3.13640308,  4.09244299,  7.69805169],
        [-2.63640308,  4.59244299,  8.19805169]]),
 array([[-3.13640308,  4.09244299,  8.19805169],
        [-2.63640308,  4.59244299,  8.69805169]]),
 array([[-3.13640308,  4.59244299,  7.69805169],
        [-2.63640308,  5.09244299,  8.19805169]]),
 array([[-2.63640308,  0.59244299,  9.69805169],
        [-2.13640308,  1.09244299, 10.19805169]]),
 array([[-2.63640308,  1.09244299,  9.69805169],
        [-2.13640308,  1.59244299, 10.19805169]]),
 array([[-2.63640308,  1.59244299,  9.19805169],
        [-2.13640308,  2.09244299,  9.69805169]]),


In [129]:
keep = (m3stop == 0)
m3AABB[keep]

array([array([[-5.13640308, -4.90755701,  3.69805169],
              [-4.63640308, -4.40755701,  4.19805169]]),
       array([[-5.13640308, -4.90755701,  4.19805169],
              [-4.63640308, -4.40755701,  4.69805169]]),
       array([[-5.13640308, -4.90755701,  4.69805169],
              [-4.63640308, -4.40755701,  5.19805169]]),
       array([[-5.13640308, -4.90755701,  5.19805169],
              [-4.63640308, -4.40755701,  5.69805169]]),
       array([[-5.13640308, -4.90755701,  5.69805169],
              [-4.63640308, -4.40755701,  6.19805169]]),
       array([[-5.13640308, -4.90755701,  6.19805169],
              [-4.63640308, -4.40755701,  6.69805169]]),
       array([[-5.13640308, -4.40755701,  3.69805169],
              [-4.63640308, -3.90755701,  4.19805169]]),
       array([[-5.13640308, -4.40755701,  4.19805169],
              [-4.63640308, -3.90755701,  4.69805169]]),
       array([[-5.13640308, -4.40755701,  4.69805169],
              [-4.63640308, -3.90755701,  5.19805

In [146]:
for i in range(8):
    print((m3stop == i).sum())


600
700
600
700
600
700
600
700


In [39]:
box_ = boxes_reached[:1000]
boxes = [box for box in box_ if pyrr.geometric_tests.ray_intersect_aabb(ray, box) is not None]

In [94]:
np.unique(idx, axis=0).shape

(5200, 3)

In [95]:
totvox/8**3

10.15625

In [96]:
8**3

512

In [9]:
voxel_size = 0.2
box = boxPR
boxes = lambda boxes_reached: [rayt.split_BB(box, voxel_size) for box in boxes_reached]
boxes_reached = np.array([boxPR])
for i in range(6):
    
    boxes_reached = np.array(boxes(boxes_reached))
    marks = [len(i) - 1 for i in boxes_reached]
    # print(marks)
    marks = np.array(marks).sum()
    print(marks)
    a,b,c,d = boxes_reached.shape
    boxes_reached = np.reshape(boxes_reached, (a*b, c, d))

    print(boxes_reached.shape)



7
(8, 2, 3)
56
(64, 2, 3)
448
(512, 2, 3)
3584
(4096, 2, 3)
28672
(32768, 2, 3)
175616


<ipython-input-9-9f8477292f07>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes_reached = np.array(boxes(boxes_reached))


ValueError: not enough values to unpack (expected 4, got 1)

In [416]:
boxes

array([[[-4.98640308, -4.75755701,  3.84805169],
        [-0.18640308,  0.04244299,  6.84805169]],

       [[-4.98640308, -4.75755701,  6.84805169],
        [-0.18640308,  0.04244299,  9.84805169]],

       [[-4.98640308,  0.04244299,  3.84805169],
        [-0.18640308,  5.04244299,  6.84805169]],

       [[-4.98640308,  0.04244299,  6.84805169],
        [-0.18640308,  5.04244299,  9.84805169]],

       [[-0.18640308, -4.75755701,  3.84805169],
        [ 4.61359692,  0.04244299,  6.84805169]],

       [[-0.18640308, -4.75755701,  6.84805169],
        [ 4.61359692,  0.04244299,  9.84805169]],

       [[-0.18640308,  0.04244299,  3.84805169],
        [ 4.61359692,  5.04244299,  6.84805169]],

       [[-0.18640308,  0.04244299,  6.84805169],
        [ 4.61359692,  5.04244299,  9.84805169]]])

In [196]:
voxel_size=0.5

# get bounding box of plant region
segtrees_dir = os.path.join(mockdir, 'toy_trees')
segtrees_files = glob.glob(os.path.join(segtrees_dir, 'tree_*.npy'))
tree = np.load(segtrees_files[0])
pointsPR = tree.T[5:8].T
pcd = lad.points2pcd(pointsPR)
voxPR = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
maxBB = voxPR.get_max_bound()
minBB = voxPR.get_min_bound()
width, height, depth = maxBB - minBB


print(type(minBB))
box = [minBB, maxBB]
voxel_size = 0.5
boxes = split_BB(box, voxel_size)
boxes


<class 'numpy.ndarray'>


[[[-5.1364030838012695, -4.907557010650635, 3.698051691055298],
  [-0.13640308380126953, 0.09244298934936523, 6.698051691055298]],
 [[-5.1364030838012695, -4.907557010650635, 6.698051691055298],
  [-0.13640308380126953, 0.09244298934936523, 10.198051691055298]],
 [[-5.1364030838012695, 0.09244298934936523, 3.698051691055298],
  [-0.13640308380126953, 5.092442989349365, 6.698051691055298]],
 [[-5.1364030838012695, 0.09244298934936523, 6.698051691055298],
  [-0.13640308380126953, 5.092442989349365, 10.198051691055298]],
 [[-0.13640308380126953, -4.907557010650635, 3.698051691055298],
  [4.8635969161987305, 0.09244298934936523, 6.698051691055298]],
 [[-0.13640308380126953, -4.907557010650635, 6.698051691055298],
  [4.8635969161987305, 0.09244298934936523, 10.198051691055298]],
 [[-0.13640308380126953, 0.09244298934936523, 3.698051691055298],
  [4.8635969161987305, 5.092442989349365, 6.698051691055298]],
 [[-0.13640308380126953, 0.09244298934936523, 6.698051691055298],
  [4.863596916198730